In [3]:
%pylab qt
from sharpy import *
from geopandas import GeoDataFrame
from ipypb import track



Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['log', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
regions = "/data/SciBigData/radargrams/nicole_thesis/chosen_area/region.gpkg"
layer = "region"

regs = GeoDataFrame.from_file(regions, layer=layer, crs=fiona.crs.from_string("+proj=stere +lat_0=90 +lat_ts=90 +lon_0=0 +k=1 +x_0=0 +y_0=0 +a=3396190 +b=3376200 +units=m +no_defs"))
regs.crs



{'proj': 'stere',
 'lat_0': 90,
 'lat_ts': 90,
 'lon_0': 0,
 'k': 1,
 'x_0': 0,
 'y_0': 0,
 'a': 3396190,
 'b': 3376200,
 'units': 'm',
 'no_defs': True}

In [6]:
folder = "/data/SciBigData/radargrams/nicole_thesis/"    

In [7]:
def file_to_tiff(filename):
    return filename[0:-9] + "_tiff.tif"



In [8]:

files = find_sharad_radargrams_in_folder(folder)
print (files)
close("all")

ts = []
for f in track(files):
    print(f"working on {f}")
    
    tab = lbl_to_geom(f)
    
    i = match_sharad_id(tab)
    t = read_geom_lbl(tab)
    t = pandas_to_geopandas_points(t)
    t = t.to_crs(regs.crs)
    lname = match_sharad_id(f)
    ts.append(t)
    t = t[t.within(regs.geometry.values[0])]
    if t.empty:
        print("skipping...")
        continue
    mx = np.max(t.radargram_column)
    mi = np.min(t.radargram_column)
    
   
    
    id_min = t.radargram_column == mi
    x_min, y_min = t[id_min].geometry.values[0].xy
    x_min, y_min = x_min[0], y_min[0]
    
    id_max = t.radargram_column == mx
    x_max, y_max = t[id_max].geometry.values[0].xy
    x_max, y_max = x_max[0], y_max[0]
    
    print(f"f: {i}: min {mi} - {(x_min, y_min)}, max {mx}- {(x_max, y_max)}, range {mx-mi}")
    
    # segy creation
    
    tiffname = f[0:-8] + "tiff.tif"
    segyname = f[0:-8] + "segy.sgy"
    # assembly the section 
    image = plt.imread(tiffname)
    newimage = crop_section(image, mi, mx)
    out=assemply_segy(newimage, x_min, x_max, y_min, y_max)
    save_segy(out, segyname)
    
    t.plot(ax=gca(), markersize=0.1)

regs.plot(ax=gca(), edgecolor="b", alpha=0.2)

['/data/SciBigData/radargrams/nicole_thesis/s_03202402_geom.lbl', '/data/SciBigData/radargrams/nicole_thesis/s_00520802_geom.lbl', '/data/SciBigData/radargrams/nicole_thesis/s_03808002_geom.lbl', '/data/SciBigData/radargrams/nicole_thesis/s_02788401_geom.lbl', '/data/SciBigData/radargrams/nicole_thesis/s_00325703_geom.lbl', '/data/SciBigData/radargrams/nicole_thesis/s_00563902_geom.lbl']


 [████████████████████████████████████████████████████████████] 6/6 [00:03<00:00, 0.43s/it]

working on /data/SciBigData/radargrams/nicole_thesis/s_03202402_geom.lbl
f: 03202402: min 2496 - (213476.76292994551, -25534.544225872367), max 3060- (365223.2493891388, -240853.2881677792), range 564
working on /data/SciBigData/radargrams/nicole_thesis/s_00520802_geom.lbl
f: 00520802: min 3378 - (313029.47216763167, -123760.84629353712), max 3605- (207875.8582362945, -137868.29695932008), range 227
working on /data/SciBigData/radargrams/nicole_thesis/s_03808002_geom.lbl
f: 03808002: min 1868 - (253214.81595754361, -37091.71288319305), max 2043- (183076.2118721235, -78907.581830567), range 175
working on /data/SciBigData/radargrams/nicole_thesis/s_02788401_geom.lbl
f: 02788401: min 2797 - (190126.06953267564, -38868.228635718), max 3237- (270901.8531968339, -227959.78114332046), range 440
working on /data/SciBigData/radargrams/nicole_thesis/s_00325703_geom.lbl
f: 00325703: min 1313 - (377753.9096147452, -217867.8222042529), max 1625- (233475.32565613222, -197659.14783661245), range 312

In [10]:
ts

[      radargram_column                     time  latitude  longitude  \
 0                    1  2013-05-26T20:28:24.566   72.8171   208.6688   
 1                    2  2013-05-26T20:28:24.680   72.8230   208.6650   
 2                    3  2013-05-26T20:28:24.840   72.8313   208.6598   
 3                    4  2013-05-26T20:28:24.977   72.8384   208.6553   
 4                    5  2013-05-26T20:28:25.137   72.8467   208.6501   
 5                    6  2013-05-26T20:28:25.274   72.8538   208.6456   
 6                    7  2013-05-26T20:28:25.434   72.8621   208.6403   
 7                    8  2013-05-26T20:28:25.571   72.8692   208.6358   
 8                    9  2013-05-26T20:28:25.731   72.8775   208.6306   
 9                   10  2013-05-26T20:28:25.868   72.8847   208.6261   
 10                  11  2013-05-26T20:28:26.028   72.8930   208.6209   
 11                  12  2013-05-26T20:28:26.165   72.9001   208.6164   
 12                  13  2013-05-26T20:28:26.303   